This notebook calculates the change in direct damage to specified assets

In [1]:
# Imports
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
import datetime
from ci_adapt_utilities import *
import pickle

In [2]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [3]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets = gpd.read_file(assets_path)
assets = gpd.GeoDataFrame(assets).set_crs(4326).to_crs(3857)
assets = assets.loc[assets.geometry.geom_type == 'LineString']
assets = assets.rename(columns={'railway' : 'asset'})

# #TODO: add reset index and test 
# # Drop passenger lines and light rails
# assets = assets.loc[~(assets['railway:traffic_mode'] == 'passenger')]
# assets = assets.loc[~(assets['asset'] == 'light_rail')]

# # Drop bridges and tunnels
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]
# assets = assets.loc[~(assets['tunnel'].isin(['yes']))]

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

# Read vulnerability and maximum damage data from Nirandjan, S., et al. (2024)
curve_types = {'rail': ['F8.1']}
infra_curves, maxdams = ds.read_vul_maxdam(data_path, hazard_type, infra_type)
max_damage_tables = pd.read_excel(data_path / vulnerability_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
print(f'Found matching infrastructure curves for: {infra_type}')

3530 assets loaded.
Found matching infrastructure curves for: rail


In [4]:
# create a variable changed_assets
changed_asset_list = [372,377,635,1358,2714] # testing with a few assets

# open pickled hazard-asset overlay and hazard intensity data
with open(interim_data_path / 'overlay_assets_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    overlay_assets = pickle.load(f)
with open(interim_data_path / 'hazard_numpified_flood_DERP_RW_L_4326_2080411370.pkl', 'rb') as f:
    hazard_numpified_list = pickle.load(f)


In [9]:
##DEV##

import geopandas as gpd
import geopandas as gpd
from pathlib import Path

def intersect_assets(protected_zone, assets):
    # Perform intersection between protected hazard zone and assets
    intersected_assets = gpd.overlay(assets, protected_zone, how='intersection')
    
    return intersected_assets
# Set up the path
adapt_path = Path(r'C:\Data\interim\adaptations\test_haz_level_adapt.geojson')

# Load the polygon
polygon = gpd.read_file(adapt_path).to_crs(3857)

filtered_assets = intersect_assets(polygon, assets)
f_assets = assets.loc[(assets['osm_id'].isin(filtered_assets['osm_id']))]
f_assets.head(3)
# assets = assets.loc[~(assets['bridge'].isin(['yes']))]



,osm_id,asset,name,gauge,electrified,voltage,bridge,maxspeed,service,tunnel,other_tags,railway:traffic_mode,usage,geometry,buffered
43,7990468,rail,None,1435,contact_line,15000,None,120,None,None,"""frequency""=>""16.7"",""operator""=>""DB Netz AG"",""...","""mixed""","""main""","LINESTRING (810131.824 6545141.628, 810131.034...","POLYGON ((810131.0332578894 6545143.573510949,..."
362,33060574,rail,Kasbachtalbahn,1435,no,None,None,None,None,None,"""embankment""=>""yes"",""operator""=>""DB Netz AG"",""...","""passenger""","""branch""","LINESTRING (808868.315 6546684.443, 808807.623...","POLYGON ((808807.6228124106 6546742.464758907,..."
364,33060576,rail,Kasbachtalbahn,1435,no,None,yes,None,None,None,"""layer""=>""1"",""operator""=>""Eifelbahn Verkehrsge...","""passenger""","""branch""","LINESTRING (808500.003 6547702.920, 808513.718...","POLYGON ((808513.716872971 6547742.594057838, ..."


In [6]:

# optionally to calculate the maximum intensity for each hazard point, this can be used, else a float can be used
max_intensity = []
for asset_id in changed_asset_list:
    max_intensity.append(retrieve_max_intensity_by_asset(asset_id, overlay_assets, hazard_numpified_list))


In [11]:

# extract the assets that will be adapted
changed_assets = assets.loc[assets.index.isin(changed_asset_list)].copy() # testing with a few assets
# add new columns fragility_mod and haz_mod
changed_assets['fragility_mod'] = 1 #[0.3, 0.5, 0.8] #fraction [example considering no reduction] (1 = no reduction, 0 = invulnerable asset) DUMMY DATA FOR TESTING
changed_assets['haz_mod'] = [np.max(x) for x in max_intensity] #meters [example adding wall of maximum flooding depth + 1 meter] (0 = no reduction in hazard intensity, 0.5 = 0.5 meter reduction in hazard intensity) DUMMY DATA FOR TESTING consider raising railway 0.5 meters

# TODO: automate infrastructure curve deduction from dictionary keys, now running with curve F8.1
hazard_intensity = infra_curves['F8.1'].index.values
fragility_values = (np.nan_to_num(infra_curves['F8.1'].values,nan=(np.nanmax(infra_curves['F8.1'].values)))).flatten()
maxdams_filt=max_damage_tables[max_damage_tables['ID number']=='F8.1']['Amount']

adaptation_run = run_damage_reduction_by_asset(geom_dict, overlay_assets, hazard_numpified_list, changed_assets, hazard_intensity, fragility_values, maxdams_filt)

#reporting
for asset_id, baseline_damages in adaptation_run[0].items():
    print(f'\nADAPTATION results for asset {asset_id}:')
    print(f'Baseline damages for asset {asset_id}: {baseline_damages[0]:.2f} to {baseline_damages[1]:.2f} EUR')
    print(f'Adapted damages for asset {asset_id}: {adaptation_run[1][asset_id][0]:.2f} to {adaptation_run[1][asset_id][1]:.2f} EUR')
    delta = tuple(adaptation_run[1][asset_id][i] - baseline_damages[i] for i in range(len(baseline_damages)))
    percent_change = tuple((100 * (delta[i] / baseline_damages[i])) for i in range(len(baseline_damages)))

    print(f'Change (Adapted-Baseline): {delta[0]:.2f} to {delta[1]:.2f} EUR, {percent_change}% change, at a cost of {adaptation_run[2][asset_id]:.2f} EUR')

#TODO Check with economist: ammortization of adaptation cost over years of adaptation scenario
    #NEXT: adaptation_run returns (collect_inb_bl, collect_inb_adapt, adaptation_cost). These can be used to calculate the EAD for the adapted scenario (and damage reduction), and compare with the adaptation cost, which must be ammortized over the years of the adaptation scenario.




2024-05-02 10:07:42 - Calculating adapted damages for assets...


100%|██████████| 100/100 [00:07<00:00, 13.34it/s]

95 assets with no change.

ADAPTATION results for asset 372:
Baseline damages for asset 372: 61372.41 to 86986.84 EUR
Adapted damages for asset 372: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -61372.41 to -86986.84 EUR, (-100.0, -100.0)% change, at a cost of 995658.76 EUR

ADAPTATION results for asset 377:
Baseline damages for asset 377: 119656.09 to 145995.72 EUR
Adapted damages for asset 377: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -119656.09 to -145995.72 EUR, (-100.0, -100.0)% change, at a cost of 1483501.51 EUR

ADAPTATION results for asset 635:
Baseline damages for asset 635: 97585.66 to 139242.60 EUR
Adapted damages for asset 635: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -97585.66 to -139242.60 EUR, (-100.0, -100.0)% change, at a cost of 1618184.76 EUR

ADAPTATION results for asset 1358:
Baseline damages for asset 1358: 2114.10 to 31020.56 EUR
Adapted damages for asset 1358: 0.00 to 0.00 EUR
Change (Adapted-Baseline): -2114.10 to -31020.56 EUR, (-100.0, -100.0)% change, 

In [ ]:
#Discussion point, logic to select the assets that will be adapted. By basin to the highest depth at a given return period (eg protect all assets in this basin for the 100 year flood event). 